This notebook is based on work by Neal Caren's ["Using Python to see how the Times writes about men and women"][1] as simplified by Bengfort, Bilbro, and Ojeda in _Applied Text Analysis with Python_ which has a [GitHub repo][2] for comparison both with Caren as well as with the current implementation.


[1]: https://nbviewer.jupyter.org/gist/nealcaren/5105037
[2]: https://github.com/foxbook/atap

In [1]:
import nltk, re, pandas as pd
from collections import Counter

In [2]:
df = pd.read_csv('../output/TEDall_speakers.csv')
print(list(df))

['Set', 'Talk_ID', 'public_url', 'headline', 'description', 'event', 'duration', 'published', 'tags', 'views', 'text', 'speaker_1', 'speaker1_occupation', 'speaker1_introduction', 'speaker1_profile', 'speaker_2', 'speaker2_occupation', 'speaker2_introduction', 'speaker2_profile', 'speaker_3', 'speaker3_occupation', 'speaker3_introduction', 'speaker3_profile', 'speaker_4', 'speaker4_occupation', 'speaker4_introduction', 'speaker4_profile']


In [3]:
texts = df.text.tolist()

So far, in working with data from our dataset, I have pulled data from a column and turned it into a list and then worked on it as a list. It makes more sense to work with the dataset directly, working through the rows using `df.itertuples()` and then `row.column_name`. 

In [4]:
for row in df.itertuples():
    print(row.public_url)

https://www.ted.com/talks/al_gore_on_averting_climate_crisis
https://www.ted.com/talks/david_pogue_says_simplicity_sells
https://www.ted.com/talks/majora_carter_s_tale_of_urban_renewal
https://www.ted.com/talks/ken_robinson_says_schools_kill_creativity
https://www.ted.com/talks/hans_rosling_shows_the_best_stats_you_ve_ever_seen
https://www.ted.com/talks/tony_robbins_asks_why_we_do_what_we_do
https://www.ted.com/talks/joshua_prince_ramus_on_seattle_s_library
https://www.ted.com/talks/julia_sweeney_on_letting_go_of_god
https://www.ted.com/talks/rick_warren_on_a_life_of_purpose
https://www.ted.com/talks/dan_dennett_s_response_to_rick_warren
https://www.ted.com/talks/cameron_sinclair_on_open_source_architecture
https://www.ted.com/talks/jehane_noujaim_inspires_a_global_day_of_film
https://www.ted.com/talks/larry_brilliant_wants_to_stop_pandemics
https://www.ted.com/talks/nicholas_negroponte_on_one_laptop_per_child
https://www.ted.com/talks/jeff_han_demos_his_breakthrough_touchscreen
https:

https://www.ted.com/talks/alessandra_orofino_it_s_our_city_let_s_fix_it
https://www.ted.com/talks/ameenah_gurib_fakim_humble_plants_that_hide_surprising_secrets
https://www.ted.com/talks/alejandro_aravena_my_architectural_philosophy_bring_the_community_into_the_process
https://www.ted.com/talks/haas_hahn_how_painting_can_transform_communities
https://www.ted.com/talks/ramanan_laxminarayan_the_coming_crisis_in_antibiotics
https://www.ted.com/talks/michael_green_what_the_social_progress_index_can_reveal_about_your_country
https://www.ted.com/talks/ethan_nadelmann_why_we_need_to_end_the_war_on_drugs
https://www.ted.com/talks/leana_wen_what_your_doctor_won_t_disclose
https://www.ted.com/talks/vincent_moon_and_nana_vasconcelos_the_world_s_hidden_music_rituals
https://www.ted.com/talks/joe_landolina_this_gel_can_make_you_stop_bleeding_instantly
https://www.ted.com/talks/rosie_king_how_autism_freed_me_to_be_myself
https://www.ted.com/talks/mark_plotkin_what_the_people_of_the_amazon_know_that_

We will need to tune this list for our corpus.

In [16]:
# Gender Word Lists

FEMININE = 'feminine'
MASCULINE = 'masculine'
UNKNOWN = 'unknown'
BOTH = 'both'

MASCULINE_WORDS = set([
    'guy','spokesman','chairman',"men's",'men','him',"he's",'his',
    'boy','boyfriend','boyfriends','boys','brother','brothers','dad',
    'dads','dude','father','fathers','fiance','gentleman','gentlemen',
    'god','grandfather','grandpa','grandson','groom','he','himself',
    'husband','husbands','king','male','man','mr','nephew','nephews',
    'priest','prince','son','sons','uncle','uncles','waiter','widower',
    'widowers'
])

FEMININE_WORDS = set([
    'heroine','spokeswoman','chairwoman',"women's",'actress','women',
    "she's",'her','aunt','aunts','bride','daughter','daughters','female',
    'fiancee','girl','girlfriend','girlfriends','girls','goddess',
    'granddaughter','grandma','grandmother','herself','ladies','lady',
    'mom','moms','mother','mothers','mrs','ms','niece','nieces',
    'priestess','princess','queens','she','sister','sisters','waitress',
    'widow','widows','wife','wives','woman'
])

In [15]:
def genderize(words):
    
    fwlen = len(FEMININE_WORDS.intersection(words))
    mwlen = len(MASCULINE_WORDS.intersection(words))
    
    if fwlen > 0 and mwlen == 0:
        return FEMININE
    elif fwlen == 0 and mwlen > 0:
        return MASCULINE
    elif fwlen > 0 and mwlen > 0:
        return BOTH
    else:
        return UNKNOWN

In [7]:
text = "My dog Molly is a good girl."
words = re.sub("[^\w+]", " ", text.lower())
print(words)

my dog molly is a good girl 


In [11]:
parse_gender(words)

100.00 female ( 1 ) 


In [9]:
def count_gender(sentences):
    
    """ REQUIRES: from collections import Counter"""
        
    sents = Counter()
    words = Counter()
    
    for sentence in sentences:
        gender = genderize(sentence)
        sents[gender] += 1
        words[gender] +=len(sentence)
        
    return sents, words

In [10]:
def parse_gender(text):
    
        """ REQUIRES: import nltk """
        
        sentences = [
            [word.lower() for word in nltk.word_tokenize(sentence)]
            for sentence in nltk.sent_tokenize(text)
        ]
        
        sents, words = count_gender(sentences)
        total = sum(words.values())
        
        for gender, count in words.items():
            pcent = (count / total) * 100
            nsents = sents[gender]
            
            print(f"{pcent:.2f} {gender} ( {nsents} ) ")

In [17]:
parse_gender(texts[0])

90.45 unknown ( 137 ) 
4.54 feminine ( 3 ) 
0.78 both ( 1 ) 
4.23 masculine ( 4 ) 


In [13]:
print(texts[0][0:100])

  Thank you so much, Chris. And it's truly a great honor to have the opportunity to come to this sta


`texts[0]` is Al Gore's talk, and so I wonder if the seeming gender neutrality here is a function of it being initially a lot of first person and then about climate change. (And, to be clear, 90% of the document is returned as "unknown", so the gendered word lists need to be examined.)

What if we did the same thing for first, second, third persons — and maybe consider singular versus plural as well?

In [14]:
for row in df.itertuples():
    gendered = parse_gender(row.text)
    print (row.public_url, gendered)

90.45 unknown ( 137 ) 
4.54 female ( 3 ) 
0.78 both ( 1 ) 
4.23 male ( 4 ) 
https://www.ted.com/talks/al_gore_on_averting_climate_crisis None
92.11 unknown ( 274 ) 
6.63 male ( 12 ) 
1.26 female ( 3 ) 
https://www.ted.com/talks/david_pogue_says_simplicity_sells None
86.61 unknown ( 170 ) 
5.18 female ( 5 ) 
7.06 male ( 13 ) 
1.15 both ( 1 ) 
https://www.ted.com/talks/majora_carter_s_tale_of_urban_renewal None
70.19 unknown ( 197 ) 
9.05 male ( 22 ) 
15.25 female ( 33 ) 
5.52 both ( 10 ) 
https://www.ted.com/talks/ken_robinson_says_schools_kill_creativity None
97.33 unknown ( 239 ) 
0.75 female ( 1 ) 
1.92 male ( 6 ) 
https://www.ted.com/talks/hans_rosling_shows_the_best_stats_you_ve_ever_seen None
85.16 unknown ( 331 ) 
8.66 male ( 25 ) 
2.10 female ( 8 ) 
4.08 both ( 8 ) 
https://www.ted.com/talks/tony_robbins_asks_why_we_do_what_we_do None
97.08 unknown ( 179 ) 
0.92 male ( 1 ) 
0.74 female ( 2 ) 
1.26 both ( 1 ) 
https://www.ted.com/talks/joshua_prince_ramus_on_seattle_s_library Non

99.25 unknown ( 117 ) 
0.75 both ( 1 ) 
https://www.ted.com/talks/frans_lanting_s_lyrical_nature_photos None
88.17 unknown ( 223 ) 
11.15 male ( 19 ) 
0.68 female ( 2 ) 
https://www.ted.com/talks/dan_dennett_on_our_consciousness None
89.03 unknown ( 186 ) 
10.11 male ( 11 ) 
0.85 female ( 1 ) 
https://www.ted.com/talks/evelyn_glennie_shows_how_to_listen None
84.66 unknown ( 9 ) 
15.34 male ( 3 ) 
https://www.ted.com/talks/caroline_lavelle_casts_a_spell_on_cello None
93.78 unknown ( 196 ) 
4.77 male ( 8 ) 
1.45 female ( 3 ) 
https://www.ted.com/talks/william_mcdonough_on_cradle_to_cradle_design None
100.00 unknown ( 7 ) 
https://www.ted.com/talks/jill_sobule_sings_to_al_gore None
91.04 unknown ( 217 ) 
5.96 male ( 8 ) 
2.13 female ( 4 ) 
0.87 both ( 2 ) 
https://www.ted.com/talks/jeff_bezos_on_the_next_web_innovation None
84.67 unknown ( 56 ) 
5.79 female ( 2 ) 
7.50 male ( 3 ) 
2.04 both ( 1 ) 
https://www.ted.com/talks/rives_remixes_ted2006 None
61.03 unknown ( 9 ) 
38.97 male ( 1 ) 


75.40 unknown ( 51 ) 
16.22 male ( 5 ) 
3.18 female ( 2 ) 
5.20 both ( 2 ) 
https://www.ted.com/talks/yossi_vardi_fights_local_warming None
92.72 unknown ( 197 ) 
6.73 female ( 11 ) 
0.55 both ( 1 ) 
https://www.ted.com/talks/deborah_gordon_digs_ants None
74.71 unknown ( 240 ) 
20.61 male ( 56 ) 
1.76 both ( 3 ) 
2.93 female ( 8 ) 
https://www.ted.com/talks/j_j_abrams_mystery_box None
74.73 unknown ( 58 ) 
3.89 female ( 1 ) 
3.45 both ( 1 ) 
17.93 male ( 11 ) 
https://www.ted.com/talks/david_gallo_shows_underwater_astonishments None
86.31 unknown ( 121 ) 
3.56 female ( 5 ) 
8.60 male ( 8 ) 
1.54 both ( 1 ) 
https://www.ted.com/talks/paola_antonelli_treats_design_as_art None
79.23 unknown ( 175 ) 
17.11 male ( 31 ) 
1.89 female ( 3 ) 
1.77 both ( 1 ) 
https://www.ted.com/talks/frank_gehry_asks_then_what None
100.00 unknown ( 74 ) 
https://www.ted.com/talks/raul_midon_plays_all_the_answers_and_tembererana None
69.24 unknown ( 224 ) 
22.63 male ( 63 ) 
6.57 female ( 16 ) 
1.56 both ( 3 ) 

59.93 unknown ( 150 ) 
34.77 male ( 74 ) 
2.94 both ( 6 ) 
2.36 female ( 6 ) 
https://www.ted.com/talks/billy_graham_on_technology_faith_and_suffering None
86.39 unknown ( 324 ) 
3.05 female ( 7 ) 
10.56 male ( 27 ) 
https://www.ted.com/talks/keith_barry_does_brain_magic None
91.73 unknown ( 191 ) 
2.94 female ( 8 ) 
4.89 male ( 9 ) 
0.43 both ( 1 ) 
https://www.ted.com/talks/martin_seligman_on_the_state_of_psychology None
42.75 unknown ( 69 ) 
17.90 female ( 26 ) 
15.99 both ( 13 ) 
23.36 male ( 31 ) 
https://www.ted.com/talks/chris_abani_muses_on_humanity None
85.42 unknown ( 104 ) 
3.00 both ( 2 ) 
1.50 female ( 2 ) 
10.08 male ( 12 ) 
https://www.ted.com/talks/louise_leakey_digs_for_humanity_s_origins None
76.04 unknown ( 201 ) 
13.80 female ( 34 ) 
9.36 male ( 21 ) 
0.81 both ( 2 ) 
https://www.ted.com/talks/jonathan_harris_collects_stories None
89.56 unknown ( 152 ) 
10.00 male ( 9 ) 
0.44 both ( 1 ) 
https://www.ted.com/talks/reed_kroloff_on_modern_and_romantic_architecture None

93.44 unknown ( 105 ) 
6.23 male ( 7 ) 
0.33 female ( 1 ) 
https://www.ted.com/talks/greg_lynn_on_organic_design None
97.84 unknown ( 111 ) 
2.16 male ( 2 ) 
https://www.ted.com/talks/rob_forbes_on_ways_of_seeing None
73.02 unknown ( 112 ) 
23.44 male ( 27 ) 
2.53 both ( 2 ) 
1.01 female ( 1 ) 
https://www.ted.com/talks/scott_mccloud_on_comics None
93.42 unknown ( 151 ) 
4.80 male ( 5 ) 
1.78 female ( 2 ) 
https://www.ted.com/talks/joseph_pine_on_what_consumers_want None
83.95 unknown ( 193 ) 
9.92 male ( 16 ) 
1.15 both ( 1 ) 
4.98 female ( 10 ) 
https://www.ted.com/talks/david_carson_on_design None
98.05 unknown ( 105 ) 
1.95 male ( 2 ) 
https://www.ted.com/talks/jamais_cascio_looks_ahead None
5.33 female ( 2 ) 
71.42 unknown ( 70 ) 
21.23 male ( 12 ) 
2.03 both ( 1 ) 
https://www.ted.com/talks/sherwin_nuland_on_hope None
93.23 unknown ( 143 ) 
5.21 male ( 5 ) 
1.56 both ( 1 ) 
https://www.ted.com/talks/woody_norris_invents_amazing_things None
91.73 unknown ( 207 ) 
0.47 both ( 1 ) 


...

83.73 unknown ( 141 ) 
16.27 male ( 23 ) 
https://www.ted.com/talks/richard_dawkins_on_our_queer_universe None
95.29 unknown ( 207 ) 
3.06 male ( 5 ) 
1.66 female ( 2 ) 
https://www.ted.com/talks/barry_schwartz_on_the_paradox_of_choice None
97.84 unknown ( 248 ) 
2.16 male ( 4 ) 
https://www.ted.com/talks/aubrey_de_grey_says_we_can_avoid_aging None
92.93 unknown ( 156 ) 
4.08 male ( 5 ) 
3.00 female ( 6 ) 
https://www.ted.com/talks/iqbal_quadir_says_mobiles_fight_poverty None
69.51 unknown ( 88 ) 
13.30 male ( 8 ) 
16.08 female ( 18 ) 
1.11 both ( 1 ) 
https://www.ted.com/talks/jacqueline_novogratz_invests_in_ending_poverty None
95.91 unknown ( 158 ) 
1.53 both ( 2 ) 
2.57 female ( 3 ) 
https://www.ted.com/talks/ashraf_ghani_on_rebuilding_broken_states None
85.81 unknown ( 168 ) 
14.19 male ( 20 ) 
https://www.ted.com/talks/sasa_vucinic_invests_in_free_press None
86.02 unknown ( 230 ) 
9.00 female ( 19 ) 
1.59 both ( 4 ) 
3.40 male ( 9 ) 
https://www.ted.com/talks/peter_donnelly_shows_

78.95 unknown ( 138 ) 
5.82 both ( 5 ) 
12.28 male ( 16 ) 
2.95 female ( 4 ) 
https://www.ted.com/talks/john_lloyd_inventories_the_invisible None
100.00 unknown ( 15 ) 
https://www.ted.com/talks/imogen_heap_wait None
91.18 unknown ( 203 ) 
8.26 male ( 10 ) 
0.56 female ( 2 ) 
https://www.ted.com/talks/jonathan_zittrain_the_web_is_a_random_act_of_kindness None
94.53 unknown ( 102 ) 
5.47 male ( 3 ) 
https://www.ted.com/talks/evgeny_morozov_is_the_internet_what_orwell_feared None
93.19 unknown ( 54 ) 
2.67 female ( 2 ) 
4.15 male ( 3 ) 
https://www.ted.com/talks/william_kamkwamba_how_i_harnessed_the_wind None
69.06 unknown ( 103 ) 
8.69 female ( 10 ) 
19.89 male ( 29 ) 
2.36 both ( 2 ) 
https://www.ted.com/talks/taryn_simon_photographs_secret_sites None
97.86 unknown ( 237 ) 
2.14 male ( 5 ) 
https://www.ted.com/talks/parag_khanna_maps_the_future_of_countries None
91.70 unknown ( 129 ) 
8.30 male ( 10 ) 
https://www.ted.com/talks/tim_brown_urges_designers_to_think_big None
87.49 unknown 

97.44 unknown ( 90 ) 
1.17 male ( 1 ) 
1.39 female ( 1 ) 
https://www.ted.com/talks/lewis_pugh_s_mind_shifting_mt_everest_swim None
97.56 unknown ( 221 ) 
0.56 female ( 1 ) 
1.89 male ( 6 ) 
https://www.ted.com/talks/jason_clay_how_big_brands_can_save_biodiversity None
42.60 unknown ( 109 ) 
8.65 male ( 13 ) 
9.19 both ( 13 ) 
39.56 female ( 74 ) 
https://www.ted.com/talks/sheryl_wudunn_our_century_s_greatest_injustice None
61.40 unknown ( 88 ) 
30.34 male ( 36 ) 
3.44 both ( 3 ) 
4.83 female ( 5 ) 
https://www.ted.com/talks/peter_molyneux_demos_milo_the_virtual_boy None
86.19 unknown ( 82 ) 
10.30 male ( 7 ) 
2.12 female ( 1 ) 
1.39 both ( 1 ) 
https://www.ted.com/talks/jamil_abu_wardeh_bringing_comedy_to_the_axis_of_evil None
72.40 unknown ( 136 ) 
27.60 male ( 35 ) 
https://www.ted.com/talks/maz_jobrani_make_jokes_not_bombs None
99.08 unknown ( 223 ) 
0.92 male ( 1 ) 
https://www.ted.com/talks/david_mccandless_the_beauty_of_data_visualization None
100.00 unknown ( 72 ) 
https://www.

9.01 female ( 5 ) 
69.16 unknown ( 61 ) 
21.84 male ( 14 ) 
https://www.ted.com/talks/elizabeth_lindsey_curating_humanity_s_heritage None
97.56 unknown ( 175 ) 
2.44 male ( 5 ) 
https://www.ted.com/talks/danny_hillis_two_frontiers_of_cancer_treatment None
82.61 unknown ( 12 ) 
17.39 male ( 2 ) 
https://www.ted.com/talks/ahn_trio_a_modern_take_on_piano_violin_cello None
58.41 unknown ( 76 ) 
4.79 male ( 4 ) 
33.35 female ( 28 ) 
3.45 both ( 2 ) 
https://www.ted.com/talks/courtney_martin_reinventing_feminism None
26.06 female ( 28 ) 
2.25 both ( 1 ) 
71.69 unknown ( 87 ) 
https://www.ted.com/talks/hans_rosling_and_the_magic_washing_machine None
68.05 unknown ( 66 ) 
28.74 male ( 22 ) 
3.22 female ( 2 ) 
https://www.ted.com/talks/mick_ebeling_the_invention_that_unlocked_a_locked_in_artist None
76.00 unknown ( 193 ) 
3.12 female ( 5 ) 
4.46 both ( 5 ) 
16.41 male ( 28 ) 
https://www.ted.com/talks/caroline_casey_looking_past_limits None
49.72 unknown ( 37 ) 
14.87 male ( 10 ) 
13.16 both ( 

KeyboardInterrupt: 